In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


test = pd.read_csv("./test.csv", index_col= 0)
train = pd.read_csv("./train.csv", index_col= 0)

In [25]:
train.corr()

C:\Users\이나라\AppData\Local\Temp\ipykernel_10316\2189804198.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,Survived,Pclass,Age,SibSp,Parch,Fare
Survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


# AGE....

In [28]:
a = train['Name'].str.contains('Miss\.')
# b = train['Name'].str.contains('Mr\.')
c = train['Name'].str.contains('Mrs\.')
d = train['Name'].str.contains('Master\.')
e = train['Name'].str.contains('Dr\.')

miss_age = train[a]['Age'].mean().__round__(1)
# mr_age = train[b]['Age'].mean().__round__(1)
mrs_age = train[c]['Age'].mean().__round__(1)
master_age = train[d]['Age'].mean().__round__(1)
dr_age = train[e]['Age'].mean().__round__(1)

# mr_t1 = train.loc[(train['Age'] >= 30) & train['Name'].str.contains('Mr\.') ]
# mr_t2 = train.loc[(train['Age'] < 30) & train['Name'].str.contains('Mr\.') ]

mr_t3 = train.loc[(train['Fare'] >= 10.5) & train['Name'].str.contains('Mr\.') & train['Age'].notnull()]
mr_t4 = train.loc[(train['Fare'] < 10.5) & train['Name'].str.contains('Mr\.') & train['Age'].notnull()]

mr_t5 =  mr_t3['Age'].mean().__round__(1)
mr_t6 =  mr_t4['Age'].mean().__round__(1)

# train.loc[train['Name'].str.contains('Mr\.'), 'Age'] = train[train['Name'].str.contains('Mr\.')]['Age'].fillna(mr_age)

train.loc[train['Name'].str.contains('Miss\.'), 'Age'] = train[train['Name'].str.contains('Miss\.')]['Age'].fillna(miss_age)
train.loc[train['Name'].str.contains('Mrs\.'), 'Age'] = train[train['Name'].str.contains('Mrs\.')]['Age'].fillna(mrs_age)
train.loc[train['Name'].str.contains('Master\.'), 'Age'] = train[train['Name'].str.contains('Master\.')]['Age'].fillna(master_age)
train.loc[(train['Name'].str.contains('Mr\.')) & train['Fare'] >= 10.5 , 'Age'] = train.loc[(train['Name'].str.contains('Mr\.')) & train['Fare'] >= 10.5 , 'Age'].fillna(mr_t5)
train.loc[(train['Name'].str.contains('Mr\.')) & train['Fare'] < 10.5 , 'Age'] = train.loc[(train['Name'].str.contains('Mr\.')) & train['Fare'] < 10.5 , 'Age'].fillna(mr_t6)
train.loc[train['Name'].str.contains('Dr\.'), 'Age'] = train[train['Name'].str.contains('Dr\.')]['Age'].fillna(mr_t5)

# test.loc[test['Name'].str.contains('Mr\.'), 'Age'] = test[test['Name'].str.contains('Mr\.')]['Age'].fillna(mr_age)

test.loc[test['Name'].str.contains('Miss\.'), 'Age'] = test[test['Name'].str.contains('Miss\.')]['Age'].fillna(miss_age)
test.loc[test['Name'].str.contains('Mrs\.'), 'Age'] = test[test['Name'].str.contains('Mrs\.')]['Age'].fillna(mrs_age)
test.loc[test['Name'].str.contains('Master\.'), 'Age'] = test[test['Name'].str.contains('Master\.')]['Age'].fillna(master_age)
test.loc[(test['Name'].str.contains('Mr\.')) & test['Fare'] >= 10.5 , 'Age'] = test.loc[(test['Name'].str.contains('Mr\.')) & test['Fare'] >= 10.5 , 'Age'].fillna(mr_t5)
test.loc[(test['Name'].str.contains('Mr\.')) & test['Fare'] < 10.5 , 'Age'] = test.loc[(test['Name'].str.contains('Mr\.')) & test['Fare'] < 10.5 , 'Age'].fillna(mr_t6)
test.loc[test['Name'].str.contains('Dr\.'), 'Age'] = test[test['Name'].str.contains('Dr\.')]['Age'].fillna(mr_t5)
test.loc[test['Name'].str.contains('Ms\.'), 'Age'] = test[test['Name'].str.contains('Ms\.')]['Age'].fillna(miss_age)

In [29]:
print(train['Age'].isnull().sum(), test['Age'].isnull().sum())

0 0


# CABIN